# Road-Segmentation
---

## Imports

In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision import transforms
from torchvision.transforms import functional as F
from PIL import Image

In [2]:
from datasets import RoadsDatasetTrain, RoadsDatasetTest
from models.unet import UNet
from models.resnet import ResNet
from train import train
from predict import predict

## Loading test and train data

In [3]:
# Neural net parameters
BATCH_SIZE = 32
EPOCHS = 1
LEARNING_RATE = 0.0001
CRITERION = nn.BCELoss()

# Image processing parameters
PATCH_SIZE = 16
LARGE_PATCH_SIZE = 96

TRAIN_IMAGE_INITIAL_SIZE = 400
NUMBER_PATCH_PER_TRAIN_IMAGE = (TRAIN_IMAGE_INITIAL_SIZE // PATCH_SIZE) * (TRAIN_IMAGE_INITIAL_SIZE // PATCH_SIZE)

TEST_IMAGE_INITIAL_SIZE = 608
NUMBER_PATCH_PER_TEST_IMAGE = (TEST_IMAGE_INITIAL_SIZE // PATCH_SIZE) * (TEST_IMAGE_INITIAL_SIZE // PATCH_SIZE)


In [4]:
# Loading train data
train_data_dir = "./Datasets/training"
train_dataset = RoadsDatasetTrain(patch_size=PATCH_SIZE, large_patch_size=LARGE_PATCH_SIZE, number_patch_per_image=NUMBER_PATCH_PER_TRAIN_IMAGE,image_initial_size= TRAIN_IMAGE_INITIAL_SIZE, root_dir=train_data_dir)
train_dataloader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Loading test data
test_data_dir = "./Datasets/test_set_images"
test_dataset = RoadsDatasetTest(patch_size=PATCH_SIZE, large_patch_size=LARGE_PATCH_SIZE, number_patch_per_image=NUMBER_PATCH_PER_TEST_IMAGE,image_initial_size= TEST_IMAGE_INITIAL_SIZE,root_dir=test_data_dir)
test_dataloader = data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

## Training our model

In [ ]:
unet = UNet()
train(model=unet, dataloader=train_dataloader, epochs=EPOCHS, criterion=CRITERION)

In [ ]:
# OPTIONAL
# Load trained model directly from file 
unet = torch.load('./Model_2', map_location=torch.device('cpu'))

## Prediction on the testing set 

In [8]:
predict(unet, test_dataloader)

CUDA is NOT available
[Patch 0/72200]


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\KaabachiBayrem\.conda\envs\ada\lib\site-packages\IPython\core\interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-38d5106b30cc>", line 1, in <module>
    predict(unet, test_dataloader)
  File "C:\Users\KaabachiBayrem\Desktop\Cours\Machine_Learning\projet_road_2\Road-Segmentation\predict.py", line 51, in predict
    for ind_batch, sample_batched in enumerate(dataloader):
  File "C:\Users\KaabachiBayrem\.conda\envs\ada\lib\site-packages\torch\utils\data\dataloader.py", line 346, in __next__
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "C:\Users\KaabachiBayrem\.conda\envs\ada\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\KaabachiBayrem\.conda\envs\ada\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in <listcomp>
 

KeyboardInterrupt: 

## Create Submission for AICrowd

In [ ]:
%run -i mask_to_submission.py